In [3]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

In [15]:
import pandas as pd


In [17]:
model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:

sequences = pipeline(
   "The car had been in the long-term lot for about two weeks, said Greg Trevor, a spokesman for the Port Authority of New York and New Jersey. What happened since Greg Trevor said?",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: The car had been in the long-term lot for about two weeks, said Greg Trevor, a spokesman for the Port Authority of New York and New Jersey. What happened since Greg Trevor said?
The Port Authority said in June that it was reviewing security and procedures for the long-term lots where some drivers were killed.
The Port Authority Police Department has been working with the New Jersey State Police to investigate a possible shooting at a long-term parking lot in Jersey City on Tuesday.


In [7]:
# Example prompt for a Q&A style
question = "What happened since Greg Trevor said, 'The car had been in the long-term lot for about two weeks'?"

sequences = pipeline(
    question,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)

# Assuming the model returns a single sequence as an answer
for seq in sequences:
    answer = seq['generated_text'].split(question)[-1]  # Extract the text after the question
    print(f"Answer: {answer.strip()}")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Answer: I'm not sure what specific incident or conversation you're referring to. Can you provide more context or information?


In [8]:
# Example of how to format your prompt to encourage a Q&A style response
question = "What happened since Greg Trevor said, 'The car had been in the long-term lot for about two weeks'?"
prompt = f"Question: {question}\nAnswer:"

sequences = pipeline(
    prompt,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)

# Assuming the model returns a single sequence that includes both the question and answer
for seq in sequences:
    # Extract everything after 'Answer:'
    answer = seq['generated_text'].split('Answer:')[-1].strip()
    print(f"Answer: {answer}")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Answer: Greg Trevor was referring to his son, Greg Jr. who passed away in a tragic accident on the side of the A-7 highway while playing basketball. Greg Jr. was driving his father's 1985 Pontiac Trans am when the car veered into a truck that was parked on the side of the highway. It is believed that Greg's death was a direct result of the head-on collision.


In [9]:
# Initial conversational context
conversation_context = """Daniel: Hello, Girafatron! How's your day going?
Girafatron: As always, I'm feeling amazing! Girafatron is always filled with positive vibes because he's always looking at the bright side of things!
Daniel: That's wonderful to hear. What have you been up to lately?
Girafatron:"""

# Generate a continuation of the conversation
sequences = pipeline(
    conversation_context,
    max_length=200,  # Adjust the length to your needs
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)

# Print the entire conversation including the newly generated continuation
for seq in sequences:
    print(seq['generated_text'])


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Daniel: Hello, Girafatron! How's your day going?
Girafatron: As always, I'm feeling amazing! Girafatron is always filled with positive vibes because he's always looking at the bright side of things!
Daniel: That's wonderful to hear. What have you been up to lately?
Girafatron: I've been keeping my schedule packed with fun activities. I'm always on the go, and I love it!
Daniel: That's great to know. Speaking of activities, what would you like to do this weekend?
Girafatron: There are so many options! Maybe we could go for a hike in the forest, take a trip to the beach, or just spend the day exploring the city.
Daniel: Those are all great ideas. Whatever you choose, I'm sure it'll be an awesome time. Can't wait to hear about


In [32]:
# Define the conversation context as a string
conversation_context = (
    "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. "
    "Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\n"
    "Daniel: Hello, Girafatron!\n"
    "Girafatron:"
)

# Generate a continuation of the conversation using the pipeline
sequences = pipeline(
    conversation_context,
    max_length=200,  # Adjust the length according to how much you want the model to generate
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)

# Output the full conversation including the model's generated continuation
for seq in sequences:
    # Append the generated text to the initial context for a full conversation
    full_conversation = conversation_context + seq['generated_text'][len(conversation_context):]
    print(full_conversation)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.
Daniel: Hello, Girafatron!
Girafatron: Hi! It's so nice to meet you finally!
Daniel: Wow, you're a tall one!
Girafatron: Thank you! I love my height. How about you, do you love animals?
Daniel: Oh, of course!
Girafatron: That's great to hear! So, do you have any favorite animals?
Daniel: Yes, I love dogs and cats.
Girafatron: That's wonderful! I am a huge fan of those animals too.
Daniel: That's interesting! What do you think about the elephant and the zebra?
Girafatron:


In [35]:

conversation_context = (
    "The car had been in the long-term lot for about two weeks, said Greg Trevor, "
    "a spokesman for the Port Authority of New York and New Jersey. "
    "What happened since Greg Trevor said?"
)

# Generate a continuation of the conversation using the pipeline
sequences = pipeline(
    conversation_context,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)

# Output the full conversation including the model's generated continuation
for seq in sequences:
    # Append the generated text to the initial context for a full conversation
    full_conversation = conversation_context + seq['generated_text'][len(conversation_context):]
    print(full_conversation)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


The car had been in the long-term lot for about two weeks, said Greg Trevor, a spokesman for the Port Authority of New York and New Jersey. What happened since Greg Trevor said?
“I am sorry for any inconvenience this may have caused you or your loved ones. If you have any further questions or concerns, please do not hesitate to ask. The Port Authority Police, Port Authority staff and our contractor will do everything possible to keep you safe and your commute on track.
“Thank you for your cooperation.”



In [38]:

conversation_context = (
    "Greg Trevor: The car had been in the long-term lot for about two weeks.\n"
    "You: What happened since Greg Trevor said?"
)

# Generate a continuation of the conversation using the pipeline
sequences = pipeline(
    conversation_context,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)

# Output the full conversation including the model's generated continuation
for seq in sequences:
    # Append the generated text to the initial context for a full conversation
    full_conversation = conversation_context + seq['generated_text'][len(conversation_context):]
    print(full_conversation)



Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Greg Trevor: The car had been in the long-term lot for about two weeks.
You: What happened since Greg Trevor said?
We are in the middle of a pandemic. We need to take precautions to protect those in high risk categories and those around us.
Greg Trevor: You need to have a mask in place. It needs to be on, and it needs to be covered up. If you are in a place where you cannot wear a mask, you need to make sure that your social distance and you're not in a position where you're going to be in contact of each other.
You need to make sure that your social distancing is taking place, because the COVID virus is very contagious.
Greg Trevor: They have been out there. They have been out of their homes for months at a time. They are getting tired, and they are getting bored. They want to get back out, but they need to protect themselves so that they can do that


In [53]:

conversation_context = (
    "The coalition won in 2002 on a wave of euphoria after 24 years of rule by Daniel Arap Moi, "
    "but now is in a precarious position because of growing public dissatisfaction. "
    "Both camps agree that the existing constitution is outdated and oppressive "
    "but have failed to reach consensus on the new one, which has been the subject of debate since 1997. "
    "What event has already finished?"
)

# Generate a continuation of the conversation using the pipeline
sequences = pipeline(
    conversation_context,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)

# Define a function to format the response with line breaks
def break_sentences(text):
    return text.replace(". ", ".\n")

# Assuming the response is a dictionary with a 'generated_text' key


# Output the conversation including the model's response
full_conversation = conversation_context + formatted_response[len(conversation_context):]
formatted_conversation = break_sentences(full_conversation)

# Print the formatted conversation
print(formatted_conversation)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


The coalition won in 2002 on a wave of euphoria after 24 years of rule by Daniel Arap Moi, but now is in a precarious position because of growing public dissatisfaction.
Both camps agree that the existing constitution is outdated and oppressive but have failed to reach consensus on the new one, which has been the subject of debate since 1997.
What event has already finished? embargo will remain in effect until Iraq pays war reparations to Kuwait to cover war damages.
What will happen while something succeeds?
If the allies succeed, Saddam Hussein will be deposed, Iraq would be under international sanctions, and Iraq's military infrastructure and its military forces would be destroyed.


In [43]:


for sample in data_samples:
    # Define the conversation context as a string, and format it like a conversation
    reporter_name = "Reporter"  # Assuming a reporter is speaking
    you_name = "You"  # The person asking the question

    conversation_context = f"{reporter_name}: {sample['Context'].strip()} {you_name}: {sample['Question']}"

    # Generate a continuation of the conversation using the pipeline
    sequences = pipeline(
        conversation_context,
        max_length=200,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        truncation=True
    )

    # Output the full conversation including the model's generated continuation
    for seq in sequences:
        # Append the generated text to the initial context for a full conversation
        full_conversation = conversation_context + seq['generated_text'][len(conversation_context):]
        print(full_conversation)
        print("\n" + "-"*50 + "\n")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Reporter: The car had been in the long-term lot for about two weeks, said Greg Trevor, a spokesman for the Port Authority of New York and New Jersey. You: What happened since Greg Trevor said?
"A man is dead after being shot to death in a 2006 Honda Civic in a lot outside of New York's Kennedy International Airport, according to a Port Authority spokesman. Port Authority officials say a man is dead after getting in a car accident outside a JFK long-term parking lot.
New York Police have said the shooting is not random."
This is so upsetting to see. My heart goes out to the family of this man. I'm so glad he got help from a stranger, but it's just sad and tragic this could have happened at such a place.
This is such a sad story. I'm so glad to hear that he got help, but it's still horrible.
This is terrible. What a horrible

--------------------------------------------------



In [45]:

conversation_context = (
    "Cuban exiles in Miami will now 'proceed with all their resources to impede or delay' the ruling, "
    "Cuba predicted Wednesday. "
    "What event has already finished?"
)

# Call the pipeline function to generate the response
response = pipeline(
    conversation_context,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)

# Define a function to format the response with line breaks
def break_sentences(text):
    return text.replace(". ", ".\n")

# Assuming the response is a dictionary with a 'generated_text' key
formatted_response = break_sentences(response[0]['generated_text'])

# Output the conversation including the model's response
full_conversation = conversation_context + formatted_response[len(conversation_context):]
print(full_conversation)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Cuban exiles in Miami will now 'proceed with all their resources to impede or delay' the ruling, Cuba predicted Wednesday. What event has already finished?
The event already finished is the Cuban exiles in Miami 'proceeding all their sources to obstruct' the ruling of a court in Cuba.
The ruling in question is an attempt to revoke the 1962 Cuban Exile Act, which has been in place for nearly five decades.
The ruling will now have to be reviewed and challenged by the Cuban government, and if successful, could be repealed.
Cuban exiles have been working to repeal the act for years.


In [52]:

conversation_context = (
    "He also referred to the Daily Mail's support for facism and its opposition to allowing Jews "
    "fleeing European pogroms to come to Britain in the 1930s. "
    "'What Mr. Livingstone was doing when he made the statement which has led to this hearing is to "
    "reflect his long-held and, it's accepted, honestly held political view of the Mail group,' "
    "Child told a disciplinary hearing in central London. "
    "What happened during the reference?"
)

# Call the pipeline function to generate the response
response = pipeline(
    conversation_context,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)

# Define a function to format the response with line breaks
def break_sentences(text):
    return text.replace(". ", ".\n")


full_conversation = conversation_context + formatted_response[len(conversation_context):]
formatted_conversation = break_sentences(full_conversation)

# Print the formatted conversation
print(formatted_conversation)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


He also referred to the Daily Mail's support for facism and its opposition to allowing Jews fleeing European pogroms to come to Britain in the 1930s.
'What Mr.
Livingstone was doing when he made the statement which has led to this hearing is to reflect his long-held and, it's accepted, honestly held political view of the Mail group,' Child told a disciplinary hearing in central London.
What happened during the reference?war reparations to Kuwait to cover war damages.
What will happen while something succeeds?
If the allies succeed, Saddam Hussein will be deposed, Iraq would be under international sanctions, and Iraq's military infrastructure and its military forces would be destroyed.


In [51]:

conversation_context = (
    "If the allies succeed, Saddam Hussein will have plunged his country first into a fruitless "
    "eight-year war against Iran and then into a humiliating war against the U.S. and the allies "
    "to defend his conquest of Kuwait, leaving much of his country's military establishment and "
    "modern infrastructure in ruins. Meanwhile, the U.S. hopes, economic sanctions and an "
    "international arms embargo will remain in effect until Iraq pays war reparations to Kuwait "
    "to cover war damages. "
    "What will happen while something succeeds?"
)

# Call the pipeline function to generate the response
response = pipeline(
    conversation_context,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)

# Define a function to format the response with line breaks
def break_sentences(text):
    return text.replace(". ", ".\n")

full_conversation = conversation_context + formatted_response[len(conversation_context):]
formatted_conversation = break_sentences(full_conversation)

# Print the formatted conversation
print(formatted_conversation)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


If the allies succeed, Saddam Hussein will have plunged his country first into a fruitless eight-year war against Iran and then into a humiliating war against the U.S.
and the allies to defend his conquest of Kuwait, leaving much of his country's military establishment and modern infrastructure in ruins.
Meanwhile, the U.S.
hopes, economic sanctions and an international arms embargo will remain in effect until Iraq pays war reparations to Kuwait to cover war damages.
What will happen while something succeeds?
If the allies succeed, Saddam Hussein will be deposed, Iraq would be under international sanctions, and Iraq's military infrastructure and its military forces would be destroyed.


In [54]:
# Define the function to break sentences
def break_sentences(text):
    # Replace each period followed by a space with a period and a newline
    return text.replace(". ", ".\n")

conversation_context = (
    "Canada's largest department store operator said the rights offering will entitle holders "
    "of its ordinary shares, except residents in the U.S. and Britain, to subscribe for two additional "
    "shares for every five shares held at a price of C$31.25 a share. The record date is Nov. 9."
)
question = "What will happen in the future?"

# Generate a continuation of the conversation using the pipeline
response = pipeline(
    "{}\n{}".format(conversation_context, question),
    max_length=200,  # Adjust the length according to how much you want the model to generate
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)

full_conversation = "{}\nAnswer: {}".format(conversation_context, response[0]['generated_text'])

# Break the conversation after each sentence
formatted_conversation = break_sentences(full_conversation)

# Print the formatted conversation
print(formatted_conversation)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Canada's largest department store operator said the rights offering will entitle holders of its ordinary shares, except residents in the U.S.
and Britain, to subscribe for two additional shares for every five shares held at a price of C$31.25 a share.
The record date is Nov.
9.
Answer: Canada's largest department store operator said the rights offering will entitle holders of its ordinary shares, except residents in the U.S.
and Britain, to subscribe for two additional shares for every five shares held at a price of C$31.25 a share.
The record date is Nov.
9.
What will happen in the future? The stock has been trading down for the last three-four days.
Will it be a short term trade? Or a long term investment?
As an investment, a lot depends on whether it is trading at a discount compared to other retailers or not.
If it is trading at a discount and you can see a long term upturn, then it may make a good long term investment if it goes down in price.
If it is trading at a discount, then 

In [55]:
# Define the function to break sentences
def break_sentences(text):
    # Replace each period followed by a space with a period and a newline
    return text.replace(". ", ".\n")

conversation_context = (
    "During the journey, Chinese and the U.S. navy held their first joint maritime exercise."
)
question = "What happened during the joint maritime exercise?"

# Generate a continuation of the conversation using the pipeline
response = pipeline(
    "{}\n{}".format(conversation_context, question),
    max_length=200,  # Adjust the length according to how much you want the model to generate
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)

full_conversation = "{}\nAnswer: {}".format(conversation_context, response[0]['generated_text'])

# Break the conversation after each sentence
formatted_conversation = break_sentences(full_conversation)

# Print the formatted conversation
print(formatted_conversation)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


During the journey, Chinese and the U.S.
navy held their first joint maritime exercise.
Answer: During the journey, Chinese and the U.S.
navy held their first joint maritime exercise.
What happened during the joint maritime exercise?
Chinese Navy and the US Navy held a joint maritime exercise in the western Pacific on 5 May 2010.
Chinese Navy's J-15 fighter jets took part in the exercise with the USS John C Stennis.


In [56]:
# Define the function to break sentences into new lines
def break_sentences(text):
    # Replace each period followed by a space with a period and a newline
    return text.replace(". ", ".\n")


conversation_context = "Li had said bluntly that it was 'the most unreasonable schedule' he had ever seen. 'Badminton competition demands physical strength,' said the head coach."

# The question from the eighth data sample
question = "What event has already finished?"

# Generate a response using the model pipeline
response = pipeline(
    f"{conversation_context}\n{question}",
    max_length=200,  # Adjust as needed
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)


full_conversation = f"Context: {conversation_context}\nQuestion: {question}\nAnswer: {response[0]['generated_text']}"

# Format the conversation by breaking it into sentences
formatted_conversation = break_sentences(full_conversation)

# Print the formatted conversation
print(formatted_conversation)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Context: Li had said bluntly that it was 'the most unreasonable schedule' he had ever seen.
'Badminton competition demands physical strength,' said the head coach.
Question: What event has already finished?
Answer: Li had said bluntly that it was 'the most unreasonable schedule' he had ever seen.
'Badminton competition demands physical strength,' said the head coach.
What event has already finished? The final, or the semi-finals?
I'm afraid I don't know.
What's your question?
The badminton events are held before and after the opening and closing ceremonies, so it's difficult to tell if a specific competition has finished yet.


In [57]:
# Define the function to break sentences into new lines
def break_sentences(text):
    # Replace each period followed by a space with a period and a newline
    return text.replace(". ", ".\n")


conversation_context = (
    "A few weeks after taking the reins of government from his brother on July 31, Raul Castro hinted "
    "at some openness to dialogue with the United States in an interview with the Communist Party newspaper Granma, "
    "suggesting ties might be normalized on equal terms. But the US had shot back that it was not talking to 'Fidel light.'"
)

# The question from the ninth data sample
question = "What event has begun but has not finished?"

# Generate a response using the model pipeline
response = pipeline(
    f"{conversation_context}\n{question}",
    max_length=200,  # Adjust as needed
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)


full_conversation = f"Context: {conversation_context}\nQuestion: {question}\nAnswer: {response[0]['generated_text']}"

# Format the conversation by breaking it into sentences
formatted_conversation = break_sentences(full_conversation)

# Print the formatted conversation
print(formatted_conversation)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Context: A few weeks after taking the reins of government from his brother on July 31, Raul Castro hinted at some openness to dialogue with the United States in an interview with the Communist Party newspaper Granma, suggesting ties might be normalized on equal terms.
But the US had shot back that it was not talking to 'Fidel light.'
Question: What event has begun but has not finished?
Answer: A few weeks after taking the reins of government from his brother on July 31, Raul Castro hinted at some openness to dialogue with the United States in an interview with the Communist Party newspaper Granma, suggesting ties might be normalized on equal terms.
But the US had shot back that it was not talking to 'Fidel light.'
What event has begun but has not finished?
It was 50 years ago that President John F.
Kennedy was killed.
(The anniversary was July 23) It took a month before Castro announced the death, and another month before the Soviet Union announced his death and released his body to pu

In [58]:
# Define the function to break sentences into new lines
def break_sentences(text):
    # Replace each period followed by a space with a period and a newline
    return text.replace(". ", ".\n")

conversation_context = (
    "A respected figure and twice a member of the Chinese People's Political Consultative Congress, "
    "Cai has used his scientific achievements to reach practical results for wetland conservation, "
    "defending the idea that 'wetlands are closely linked to human survival and development.' "
    "Cai also stressed the importance of wetlands for sustainable economic development and human well-being, "
    "especially in their function as the 'kidneys of nature.'"
)

# The question from the tenth data sample
question = "What happened before conservation?"

# Generate a response using the model pipeline
response = pipeline(
    f"{conversation_context}\n{question}",
    max_length=200,  # Adjust as needed
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)


full_conversation = f"Context: {conversation_context}\nQuestion: {question}\nAnswer: {response[0]['generated_text']}"

# Format the conversation by breaking it into sentences
formatted_conversation = break_sentences(full_conversation)

# Print the formatted conversation
print(formatted_conversation)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Context: A respected figure and twice a member of the Chinese People's Political Consultative Congress, Cai has used his scientific achievements to reach practical results for wetland conservation, defending the idea that 'wetlands are closely linked to human survival and development.' Cai also stressed the importance of wetlands for sustainable economic development and human well-being, especially in their function as the 'kidneys of nature.'
Question: What happened before conservation?
Answer: A respected figure and twice a member of the Chinese People's Political Consultative Congress, Cai has used his scientific achievements to reach practical results for wetland conservation, defending the idea that 'wetlands are closely linked to human survival and development.' Cai also stressed the importance of wetlands for sustainable economic development and human well-being, especially in their function as the 'kidneys of nature.'
What happened before conservation?
In the early stages befor

In [59]:
# Define the function to break sentences into new lines
def break_sentences(text):
    # Replace each period followed by a space with a period and a newline
    return text.replace(". ", ".\n")

conversation_context = (
    "The combined effect of these changes is expected to save the company about $4 million in interest expenses next year, "
    "or six cents a share. Shoney's said the latest restructuring affected bank indebtedness that was incurred to finance "
    "$585 million of the company's $728 million recapitalization that took place in 1988."
)

# The question from the eleventh data sample
question = "What happened before restructuring?"

# Generate a response using the model pipeline
response = pipeline(
    f"{conversation_context}\n{question}",
    max_length=200,  # Adjust as needed
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True
)


full_conversation = f"Context: {conversation_context}\nQuestion: {question}\nAnswer: {response[0]['generated_text']}"

# Format the conversation by breaking it into sentences
formatted_conversation = break_sentences(full_conversation)

# Print the formatted conversation
print(formatted_conversation)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Context: The combined effect of these changes is expected to save the company about $4 million in interest expenses next year, or six cents a share.
Shoney's said the latest restructuring affected bank indebtedness that was incurred to finance $585 million of the company's $728 million recapitalization that took place in 1988.
Question: What happened before restructuring?
Answer: The combined effect of these changes is expected to save the company about $4 million in interest expenses next year, or six cents a share.
Shoney's said the latest restructuring affected bank indebtedness that was incurred to finance $585 million of the company's $728 million recapitalization that took place in 1988.
What happened before restructuring? The company had been unable to repay a $585 million bank loan, and had been forced to undergo a reorganization to prevent default on the debt.
As part of that, Shoney closed 27 of its 42 outlets.
Shoney's has a $4 million line of credit from its bank, which has